In [ ]:
!wget https://raw.githubusercontent.com/iabufarha/iSarcasmEval/main/train/train.En.csv
!wget https://raw.githubusercontent.com/iabufarha/iSarcasmEval/main/test/task_A_En_test.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
root_dir = "/content/drive/My Drive/"
project_folder = "IFT-6759B"

os.chdir(root_dir + project_folder)

mut001=pd.read_csv("mutation_001.csv")
mut010=pd.read_csv("mutation_010.csv")
mut011=pd.read_csv("mutation_011.csv")
mut100=pd.read_csv("mutation_100.csv")
mut101=pd.read_csv("mutation_101.csv")
mut110=pd.read_csv("mutation_110.csv")
mut111=pd.read_csv("mutation_111.csv")

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import Trainer

class SarcasmDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
## Test Dataset
class SarcasmTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(labels, pred)

    return {"accuracy": accuracy,"f1_score":f1}

def labels(x):
    if x == 0:
        return 0
    else:
        return 1

## BERT-base

In [ ]:
path = 'train.En.csv'
path_test = 'task_A_En_test.csv'

df = pd.read_csv(path)
test = pd.read_csv(path_test)
df = df.dropna(subset=['tweet'])

train = df

train_tweets = train['tweet'].values.tolist()
train_labels = train['sarcastic'].values.tolist()
test_tweets = test['text'].values.tolist()
test_labels = test['sarcastic'].values.tolist()

train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets, train_labels, 
                                                                    test_size=0.1,random_state=42,stratify=train_labels)
model_name = 'detecting-Sarcasm'


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', num_labels=2,
                                        loss_function_params={"weight": [0.75, 0.25]})


train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_tweets, padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.573900,0.632263,0.636888,0.411215
1000,0.397900,0.858024,0.752161,0.358209


{'eval_loss': 0.6322631239891052,
 'eval_accuracy': 0.6368876080691642,
 'eval_f1_score': 0.411214953271028,
 'eval_runtime': 3.7454,
 'eval_samples_per_second': 92.647,
 'eval_steps_per_second': 11.748,
 'epoch': 3.0}

In [ ]:
test_tweets = test['text'].values.tolist() 
test_labels = test['sarcastic'].values.tolist() 
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

              precision    recall  f1-score   support

           0       0.91      0.60      0.72      1200
           1       0.21      0.65      0.32       200

    accuracy                           0.61      1400
   macro avg       0.56      0.62      0.52      1400
weighted avg       0.81      0.61      0.66      1400



### With simple augmentation (mutation-based)

In [ ]:
path = 'train.En.csv'
path_test = 'task_A_En_test.csv'

df = pd.read_csv(path)
test = pd.read_csv(path_test)
df = df.dropna(subset=['tweet'])

train = df

train_tweets = train['tweet'].values.tolist()
train_labels = train['sarcastic'].values.tolist()
test_tweets = test['text'].values.tolist()
test_labels = test['sarcastic'].values.tolist()

train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets, train_labels, 
                                                                    test_size=0.1,random_state=42,stratify=train_labels)
model_name = 'detecting-Sarcasm'

train_tweets = [*train_tweets, *mut101["tweet"].values.tolist()]
train_labels = [*train_labels,*mut101["sarcastic"].values.tolist()]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', num_labels=2,
                                        loss_function_params={"weight": [0.75, 0.25]})


train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_tweets, padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.541600,0.656328,0.717579,0.279412
1000,0.416900,0.744831,0.755043,0.421769


{'eval_loss': 0.6563280820846558,
 'eval_accuracy': 0.7175792507204611,
 'eval_f1_score': 0.27941176470588236,
 'eval_runtime': 2.3148,
 'eval_samples_per_second': 149.905,
 'eval_steps_per_second': 19.008,
 'epoch': 3.0}

In [ ]:
test_tweets = test['text'].values.tolist() 
test_labels = test['sarcastic'].values.tolist() 
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83      1200
           1       0.12      0.15      0.13       200

    accuracy                           0.71      1400
   macro avg       0.48      0.48      0.48      1400
weighted avg       0.75      0.71      0.73      1400



## RoBERTa

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.3]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

train = pd.read_csv('/content/train.En.csv')
test = pd.read_csv('/content/task_A_En_test.csv')

train_tweets = train['tweet'].astype(str).values.tolist()
train_labels = train['sarcastic'].values.tolist()
test_tweets = test['text'].astype(str).values.tolist()
test_labels = test['sarcastic'].values.tolist()

model_name = 'detecting-Sarcasm'

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, num_labels=2, 
                                          loss_function_params={"weight": [0.75, 0.25]})

train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

training_args = TrainingArguments(
                      output_dir='./', 
                      evaluation_strategy="steps", 
                      num_train_epochs=5, 
                      per_device_train_batch_size=32,
                      per_device_eval_batch_size=64, 
                      warmup_steps=500, weight_decay=0.01,
                      logging_dir='./logs4', load_best_model_at_end=True)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

trainer = Trainer(model=model, 
                  args=training_args, 
                  train_dataset=train_dataset,
                  eval_dataset=test_dataset,
                  compute_metrics=compute_metrics)

trainer.train()
trainer.evaluate()

test_tweets = test['text'].values.tolist() 
test_labels = test['sarcastic'].values.tolist() 
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.474900,1.160883,0.680714,0.345534


{'eval_loss': 1.160882830619812,
 'eval_accuracy': 0.6807142857142857,
 'eval_f1_score': 0.3455344070278184,
 'eval_runtime': 10.8187,
 'eval_samples_per_second': 129.406,
 'eval_steps_per_second': 2.034,
 'epoch': 5.0}

In [ ]:
test_tweets = test['text'].values.tolist() 
test_labels = test['sarcastic'].values.tolist() 
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

              precision    recall  f1-score   support

           0       0.91      0.70      0.79      1200
           1       0.24      0.59      0.35       200

    accuracy                           0.68      1400
   macro avg       0.58      0.64      0.57      1400
weighted avg       0.82      0.68      0.73      1400



### With simple data augmentation (mutation-based)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.3]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

train = pd.read_csv('train.En.csv')
test = pd.read_csv('task_A_En_test.csv')

train_tweets = train['tweet'].astype(str).values.tolist()
train_labels = train['sarcastic'].values.tolist()
test_tweets = test['text'].astype(str).values.tolist()
test_labels = test['sarcastic'].values.tolist()

train_tweets = [*train_tweets, *mut101["tweet"].astype(str).values.tolist()]
train_labels = [*train_labels,*mut101["sarcastic"].values.tolist()]

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, num_labels=2, loss_function_params={"weight": [0.75, 0.25]})

train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

training_args = TrainingArguments(output_dir='./', 
                                  evaluation_strategy="steps", 
                                  num_train_epochs=5, 
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=64, 
                                  warmup_steps=500, weight_decay=0.01,
                                  logging_dir='./logs4', load_best_model_at_end=True)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

model.save_pretrained(MODEL)

trainer = Trainer(
                model=model, args=training_args, train_dataset=train_dataset,
                eval_dataset=test_dataset,
                compute_metrics=compute_metrics)

trainer.train()
trainer.evaluate()

test_tweets = test['text'].values.tolist() 
test_labels = test['sarcastic'].values.tolist() 
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.460600,0.675183,0.765000,0.402904


{'eval_loss': 0.6751834750175476,
 'eval_accuracy': 0.765,
 'eval_f1_score': 0.4029038112522686,
 'eval_runtime': 9.9878,
 'eval_samples_per_second': 140.171,
 'eval_steps_per_second': 2.203,
 'epoch': 5.0}

In [ ]:
test_tweets = test['text'].values.tolist() 
test_labels = test['sarcastic'].values.tolist() 
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

              precision    recall  f1-score   support

           0       0.92      0.80      0.85      1200
           1       0.32      0.56      0.40       200

    accuracy                           0.77      1400
   macro avg       0.62      0.68      0.63      1400
weighted avg       0.83      0.77      0.79      1400



## XLNet

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.utils import shuffle
from transformers import XLNetTokenizer
from transformers import XLNetForSequenceClassification

train = pd.read_csv('train.En.csv')
test = pd.read_csv('task_A_En_test.csv')

X_train = train['tweet'].astype('str').values.tolist()
y_train = train['sarcastic'].values.tolist()
X_test = test['text'].astype('str').values.tolist()
y_test = test['sarcastic'].values.tolist()

# train_tweets = [*train_tweets, *mut101["tweet"].astype(str).values.tolist()]
# train_labels = [*train_labels,*mut101["sarcastic"].values.tolist()]

model_name = 'detecting-sarcasm'
task='sentiment'
MODEL = 'xlnet-base-cased'

tokenizer = XLNetTokenizer.from_pretrained(MODEL, num_labels=2, loss_function_params={"weight": [0.75, 0.25]})

train_encodings = tokenizer(X_train, truncation=True, padding=True,return_tensors = 'pt')
test_encodings = tokenizer(X_test,truncation=True, padding=True,return_tensors = 'pt')

train_dataset = SarcasmDataset(train_encodings, y_train)
test_dataset = SarcasmDataset(test_encodings, y_test)

training_args = TrainingArguments(output_dir='./res', 
                                  num_train_epochs=5, 
                                  per_device_train_batch_size=32, 
                                  warmup_steps=500, 
                                  weight_decay=0.01,
                                  logging_dir='./logs4')

model = XLNetForSequenceClassification.from_pretrained(MODEL)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
)
trainer.train()

preds = trainer.predict(test_dataset)
preds = np.argmax(preds.predictions[:, 0:2], axis=-1)
print(classification_report(y_test, preds))
print(f1_score(y_test, preds))

## Electra

In [ ]:
!pip install torchmetrics pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.4 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torch.optim import AdamW
import torchmetrics

import pytorch_lightning as pl
from pytorch_lightning import Trainer

from typing import Optional

from transformers import ElectraModel, ElectraPreTrainedModel, ElectraTokenizerFast as ElectraTokenizer
from transformers.models.electra.modeling_electra import ElectraClassificationHead

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

pl.seed_everything(42)

class TweetDataset(Dataset):
    def __init__(self, data:pd.DataFrame, tokenizer: ElectraTokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        encoding = self.tokenizer(
            row.tweet,
            max_length = 64,
            truncation = True,
            padding = "max_length",
            add_special_tokens = True,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors = "pt"
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(row.sarcastic)
        }

class TweetDataModule(pl.LightningDataModule):
    def __init__(self, tokenizer: ElectraTokenizer, batch_size: int):
        self.tokenizer = tokenizer
        self.batch_size = batch_size

        self.setup()
    
    def setup(self, stage: Optional[str] = None):
        ### add for loading 
        train_df = pd.read_csv('/content/train.En.csv')[['tweet', 'sarcastic']]
        train_df = train_df[train_df['tweet'].notna()]
        self.train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

        test_df = pd.read_csv('/content/task_A_En_test.csv')[['text', 'sarcastic']]
        test_df = test_df[test_df['text'].notna()]
        test_df['tweet'] = test_df['text']
        self.test_df = test_df

        self.train_df, self.val_df = train_test_split(self.train_df, test_size=0.1)
    
    def train_dataloader(self):
        return DataLoader(
            dataset = TweetDataset(self.train_df, self.tokenizer),
            batch_size = self.batch_size,
            num_workers = os.cpu_count(),
            shuffle = True
        )

    def val_dataloader(self):
        return DataLoader(
            dataset = TweetDataset(self.val_df, self.tokenizer),
            batch_size = self.batch_size,
            num_workers = os.cpu_count(),
            shuffle = False
        )  

    def test_dataloader(self):
        return DataLoader(
            dataset = TweetDataset(self.test_df, self.tokenizer),
            batch_size = self.batch_size,
            num_workers = os.cpu_count(),
            shuffle = False
        )

class ElectraClassifier(ElectraPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.n_classes = config.num_labels
        self.config = config
        self.electra = ElectraModel(config)
        self.classifier = ElectraClassificationHead(config)

        self.post_init()

    def forward(self, input_ids = None, attention_mask = None):
        discriminator_hidden_states = self.electra(input_ids, attention_mask)
        sequence_output = discriminator_hidden_states[0]
        logits = self.classifier(sequence_output)
        return logits

class SarcasmClassifier(pl.LightningModule):
    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes

        self.classifier = ElectraClassifier.from_pretrained("google/electra-small-discriminator", num_labels = n_classes)

        class_weights = torch.FloatTensor([1, 3]).cuda()
        self.criterion = nn.CrossEntropyLoss(weight=class_weights)

        self.logits = None
        self.preds = []
    
    def forward(self, input_ids, attention_mask):
        return self.classifier(input_ids, attention_mask)
    
    def run_step(self, batch, stage):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"].long()
        logits = self(input_ids, attention_mask)

        self.logits = logits

        loss = self.criterion(logits, labels)

        self.log(f"{stage}_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def training_step(self, batch, batch_idx):
        return self.run_step(batch, "train")
    
    def validation_step(self, batch, batch_idx):
        return self.run_step(batch, "val")
    
    def test_step(self, batch, batch_idx):
        r = self.run_step(batch, "test")
        self.preds += list(self.logits.cpu().data.numpy().argmax(axis=1))
        return r

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-4)
    
MODEL_NAME = "google/electra-base-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)

data_module = TweetDataModule(tokenizer, batch_size = 32)

clf = SarcasmClassifier(2)
trainer = Trainer(max_epochs=20, devices=1, accelerator="gpu", log_every_n_steps=1)

trainer.fit(clf, data_module.train_dataloader(), data_module.val_dataloader())

trainer.test(clf, data_module.test_dataloader())

y_pred = clf.preds

y_test = []
for i in data_module.test_dataloader().dataset:
    y_test.append(i['label'].data.numpy())

print(f1_score(y_test, y_pred))

INFO:lightning_fabric.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_loss_epoch        1.9657065868377686
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
0.3097643097643098


### With simple augmentation (mutation-based)

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torch.optim import AdamW
import torchmetrics

import pytorch_lightning as pl
from pytorch_lightning import Trainer

from typing import Optional

from transformers import ElectraModel, ElectraPreTrainedModel, ElectraTokenizerFast as ElectraTokenizer
from transformers.models.electra.modeling_electra import ElectraClassificationHead

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

pl.seed_everything(42)

class TweetDataset(Dataset):
    def __init__(self, data:pd.DataFrame, tokenizer: ElectraTokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        encoding = self.tokenizer(
            row.tweet,
            max_length = 64,
            truncation = True,
            padding = "max_length",
            add_special_tokens = True,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors = "pt"
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(row.sarcastic)
        }

class TweetDataModule(pl.LightningDataModule):
    def __init__(self, tokenizer: ElectraTokenizer, batch_size: int):
        self.tokenizer = tokenizer
        self.batch_size = batch_size

        self.setup()
    
    def setup(self, stage: Optional[str] = None):
        ### add for loading 
        train_df = pd.read_csv('train.En.csv')[['tweet', 'sarcastic']]
        train_df = train_df[train_df['tweet'].notna()]
        self.train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

        test_df = pd.read_csv('task_A_En_test.csv')[['text', 'sarcastic']]
        test_df = test_df[test_df['text'].notna()]
        test_df['tweet'] = test_df['text']
        self.test_df = test_df

        self.train_df, self.val_df = train_test_split(self.train_df, test_size=0.1)
        self.train_df = self.train_df[["tweet","sarcastic"]].append(mut101[["tweet","sarcastic"]])
    
    def train_dataloader(self):
        return DataLoader(
            dataset = TweetDataset(self.train_df, self.tokenizer),
            batch_size = self.batch_size,
            num_workers = os.cpu_count(),
            shuffle = True
        )

    def val_dataloader(self):
        return DataLoader(
            dataset = TweetDataset(self.val_df, self.tokenizer),
            batch_size = self.batch_size,
            num_workers = os.cpu_count(),
            shuffle = False
        )  

    def test_dataloader(self):
        return DataLoader(
            dataset = TweetDataset(self.test_df, self.tokenizer),
            batch_size = self.batch_size,
            num_workers = os.cpu_count(),
            shuffle = False
        )

class ElectraClassifier(ElectraPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.n_classes = config.num_labels
        self.config = config
        self.electra = ElectraModel(config)
        self.classifier = ElectraClassificationHead(config)

        self.post_init()

    def forward(self, input_ids = None, attention_mask = None):
        discriminator_hidden_states = self.electra(input_ids, attention_mask)
        sequence_output = discriminator_hidden_states[0]
        logits = self.classifier(sequence_output)
        return logits

class SarcasmClassifier(pl.LightningModule):
    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes

        self.classifier = ElectraClassifier.from_pretrained("google/electra-small-discriminator", num_labels = n_classes)

        class_weights = torch.FloatTensor([1, 3]).cuda()
        self.criterion = nn.CrossEntropyLoss(weight=class_weights)

        self.logits = None
        self.preds = []
    
    def forward(self, input_ids, attention_mask):
        return self.classifier(input_ids, attention_mask)
    
    def run_step(self, batch, stage):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"].long()
        logits = self(input_ids, attention_mask)

        self.logits = logits

        loss = self.criterion(logits, labels)

        self.log(f"{stage}_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def training_step(self, batch, batch_idx):
        return self.run_step(batch, "train")
    
    def validation_step(self, batch, batch_idx):
        return self.run_step(batch, "val")
    
    def test_step(self, batch, batch_idx):
        r = self.run_step(batch, "test")
        self.preds += list(self.logits.cpu().data.numpy().argmax(axis=1))
        return r

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-4)
    
MODEL_NAME = "google/electra-base-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)

data_module = TweetDataModule(tokenizer, batch_size = 32)

clf = SarcasmClassifier(2)
trainer = Trainer(max_epochs=20, devices=1, accelerator="gpu", log_every_n_steps=1)

trainer.fit(clf, data_module.train_dataloader(), data_module.val_dataloader())

trainer.test(clf, data_module.test_dataloader())

y_pred = clf.preds

y_test = []
for i in data_module.test_dataloader().dataset:
    y_test.append(i['label'].data.numpy())

print(f1_score(y_test, y_pred))

INFO:lightning_fabric.utilities.seed:Global seed set to 42


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraClassifier: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraClassifier were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_loss_epoch      │    1.9745774269104004     │
└───────────────────────────┴───────────────────────────┘

0.23582089552238805


## DeBERTa (large is too slow to train)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.3]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

train = pd.read_csv('/content/train.En.csv')
test = pd.read_csv('/content/task_A_En_test.csv')

train_tweets = train['tweet'].astype(str).values.tolist()
train_labels = train['sarcastic'].values.tolist()
test_tweets = test['text'].astype(str).values.tolist()
test_labels = test['sarcastic'].values.tolist()

model_name = 'detecting-Sarcasm'

task='sentiment'
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, num_labels=2, loss_function_params={"weight": [0.75, 0.25]})

train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir='./res', evaluation_strategy="steps", num_train_epochs=5, per_device_train_batch_size=16,
    per_device_eval_batch_size=64, warmup_steps=500, weight_decay=0.01,logging_dir='./logs4',
    load_best_model_at_end=True,
)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.571400,0.471265,0.797143,0.330189
1000,0.206100,1.118623,0.773571,0.332632


{'eval_loss': 0.47126534581184387,
 'eval_accuracy': 0.7971428571428572,
 'eval_f1_score': 0.33018867924528306,
 'eval_runtime': 11.7941,
 'eval_samples_per_second': 118.703,
 'eval_steps_per_second': 1.865,
 'epoch': 5.0}

In [ ]:
test_tweets = test['text'].values.tolist() 
test_labels = test['sarcastic'].values.tolist() 
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      1200
           1       0.31      0.35      0.33       200

    accuracy                           0.80      1400
   macro avg       0.60      0.61      0.61      1400
weighted avg       0.81      0.80      0.80      1400



### With simple augmentation (mutation-based)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.3]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

train = pd.read_csv('train.En.csv')
test = pd.read_csv('task_A_En_test.csv')

train_tweets = train['tweet'].astype(str).values.tolist()
train_labels = train['sarcastic'].values.tolist()
test_tweets = test['text'].astype(str).values.tolist()
test_labels = test['sarcastic'].values.tolist()

train_tweets = [*train_tweets, *mut101["tweet"].astype(str).values.tolist()]
train_labels = [*train_labels,*mut101["sarcastic"].values.tolist()]

model_name = 'detecting-Sarcasm'

task='sentiment'
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, num_labels=2, loss_function_params={"weight": [0.75, 0.25]})

train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir='./res', evaluation_strategy="steps", num_train_epochs=5, per_device_train_batch_size=16,
    per_device_eval_batch_size=64, warmup_steps=500, weight_decay=0.01,logging_dir='./logs4',
    load_best_model_at_end=True,
)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# model.save_pretrained(MODEL)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.544200,0.470425,0.808571,0.279570
1000,0.227500,0.825628,0.820714,0.319783


{'eval_loss': 0.4704248011112213,
 'eval_accuracy': 0.8085714285714286,
 'eval_f1_score': 0.27956989247311825,
 'eval_runtime': 10.9394,
 'eval_samples_per_second': 127.978,
 'eval_steps_per_second': 2.011,
 'epoch': 5.0}

In [ ]:
test_tweets = test['text'].values.tolist() 
test_labels = test['sarcastic'].values.tolist() 
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      1200
           1       0.30      0.26      0.28       200

    accuracy                           0.81      1400
   macro avg       0.59      0.58      0.58      1400
weighted avg       0.80      0.81      0.80      1400



## Ensemble

In [ ]:
path = '/content/train.En.csv'
path_test = '/content/task_A_En_test.csv'

train = pd.read_csv(path)
test = pd.read_csv(path_test)
train = train.dropna(subset=['tweet'])

train_tweets = train['tweet'].values.tolist()
train_labels = train['sarcastic'].values.tolist()
test_tweets = test['text'].values.tolist()
test_labels = test['sarcastic'].values.tolist()

model_dict = {'cardiffnlp/twitter-roberta-base-sentiment':[AutoTokenizer, AutoModelForSequenceClassification],
              'xlnet-base-cased':[XLNetTokenizer, XLNetForSequenceClassification],
              'bert-base-uncased':[BertTokenizer, BertForSequenceClassification]}

hard_predictions = []
soft_predictions = []

for key, val in model_dict.items():
  print(key)
  tokenizer = val[0].from_pretrained(key, num_labels=2,
                                          loss_function_params={"weight": [0.75, 0.25]})


  train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512)
  test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

  train_dataset = SarcasmDataset(train_encodings, train_labels)
  test_dataset = SarcasmDataset(test_encodings, test_labels)

  if key=='xlnet-base-cased':
    training_args = TrainingArguments(output_dir='./res', 
                                      num_train_epochs=5, 
                                      per_device_train_batch_size=32, 
                                      warmup_steps=500, 
                                      weight_decay=0.01,
                                      logging_dir='./logs4')
  elif key=='cardiffnlp/twitter-roberta-base-sentiment':
    training_args = TrainingArguments(output_dir='./res', 
                                      evaluation_strategy="steps", 
                                      num_train_epochs=5, 
                                      per_device_train_batch_size=32,
                                      per_device_eval_batch_size=64, 
                                      warmup_steps=500, 
                                      weight_decay=0.01,
                                      logging_dir='./logs4',
                                      load_best_model_at_end=True)
  else:
    training_args = TrainingArguments(output_dir="output",
                                      evaluation_strategy="steps",
                                      eval_steps=500,
                                      per_device_train_batch_size=8,
                                      per_device_eval_batch_size=8,
                                      num_train_epochs=5,
                                      seed=0,
                                      load_best_model_at_end=True)

  
  if key=='bert-base-uncased':
    model = val[1].from_pretrained(key,num_labels=2)
  else:
    model = val[1].from_pretrained(key)

  trainer = Trainer(
      model=model, args=training_args, 
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
      compute_metrics=compute_metrics,
  )

  trainer.train()

  preds = trainer.predict(test_dataset=test_dataset)

  if key=='xlnet-base-cased':
    preds = trainer.predict(test_dataset)
    preds = preds.predictions[:, 0:2]
    results = np.argmax(preds, axis=-1)
  else:
    probs = torch.from_numpy(preds[0]).softmax(1)
    preds = probs.numpy()
    results = np.argmax(preds,axis=1)

  print(classification_report(test_labels, results))

  hard_predictions.append(results)
  soft_predictions.append(preds)
  

cardiffnlp/twitter-roberta-base-sentiment


Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.468200,0.928753,0.762143,0.393443


              precision    recall  f1-score   support

           0       0.91      0.80      0.85      1200
           1       0.31      0.54      0.39       200

    accuracy                           0.76      1400
   macro avg       0.61      0.67      0.62      1400
weighted avg       0.83      0.76      0.79      1400

xlnet-base-cased


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Step,Training Loss
500,0.531700


              precision    recall  f1-score   support

           0       0.89      0.88      0.88      1200
           1       0.31      0.33      0.32       200

    accuracy                           0.80      1400
   macro avg       0.60      0.60      0.60      1400
weighted avg       0.81      0.80      0.80      1400

bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,0.570200,0.471778,0.862857,0.142857
1000,0.496000,0.660340,0.741429,0.276000
1500,0.253100,1.104155,0.806429,0.245125
2000,0.091200,1.384288,0.757143,0.294606


              precision    recall  f1-score   support

           0       0.87      0.99      0.93      1200
           1       0.67      0.08      0.14       200

    accuracy                           0.86      1400
   macro avg       0.77      0.54      0.53      1400
weighted avg       0.84      0.86      0.81      1400



In [ ]:
binary_soft_predictions = (soft_predictions[0][:,:-1]+soft_predictions[1]+soft_predictions[2])/3
binary_hard_predictions = np.array(np.mean(np.array(hard_predictions), axis=0) > 0.5).astype(int)

print(classification_report(test_labels, np.argmax(binary_soft_predictions, axis=1)))
print(classification_report(test_labels, binary_hard_predictions))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89      1200
           1       0.36      0.35      0.36       200

    accuracy                           0.82      1400
   macro avg       0.63      0.63      0.63      1400
weighted avg       0.82      0.82      0.82      1400

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      1200
           1       0.44      0.29      0.35       200

    accuracy                           0.85      1400
   macro avg       0.67      0.62      0.63      1400
weighted avg       0.83      0.85      0.83      1400



### With simple augmentation (mutation-based)

In [ ]:
path = 'train.En.csv'
path_test = 'task_A_En_test.csv'

train = pd.read_csv(path)
test = pd.read_csv(path_test)
train = train.dropna(subset=['tweet'])

train_tweets = train['tweet'].values.tolist()
train_labels = train['sarcastic'].values.tolist()
test_tweets = test['text'].values.tolist()
test_labels = test['sarcastic'].values.tolist()

train_tweets = [*train_tweets, *mut101["tweet"].astype(str).values.tolist()]
train_labels = [*train_labels,*mut101["sarcastic"].values.tolist()]

model_dict = {'cardiffnlp/twitter-roberta-base-sentiment':[AutoTokenizer, AutoModelForSequenceClassification],
              'xlnet-base-cased':[XLNetTokenizer, XLNetForSequenceClassification],
              'bert-base-uncased':[BertTokenizer, BertForSequenceClassification]}

hard_predictions = []
soft_predictions = []

for key, val in model_dict.items():
  print(key)
  tokenizer = val[0].from_pretrained(key, num_labels=2,
                                          loss_function_params={"weight": [0.75, 0.25]})


  train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512)
  test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

  train_dataset = SarcasmDataset(train_encodings, train_labels)
  test_dataset = SarcasmDataset(test_encodings, test_labels)

  if key=='xlnet-base-cased':
    training_args = TrainingArguments(output_dir='./res', 
                                      num_train_epochs=5, 
                                      per_device_train_batch_size=32, 
                                      warmup_steps=500, 
                                      weight_decay=0.01,
                                      logging_dir='./logs4')
  elif key=='cardiffnlp/twitter-roberta-base-sentiment':
    training_args = TrainingArguments(output_dir='./res', 
                                      evaluation_strategy="steps", 
                                      num_train_epochs=5, 
                                      per_device_train_batch_size=32,
                                      per_device_eval_batch_size=64, 
                                      warmup_steps=500, 
                                      weight_decay=0.01,
                                      logging_dir='./logs4',
                                      load_best_model_at_end=True)
  else:
    training_args = TrainingArguments(output_dir="output",
                                      evaluation_strategy="steps",
                                      eval_steps=500,
                                      per_device_train_batch_size=8,
                                      per_device_eval_batch_size=8,
                                      num_train_epochs=5,
                                      seed=0,
                                      load_best_model_at_end=True)

  
  if key=='bert-base-uncased':
    model = val[1].from_pretrained(key,num_labels=2)
  else:
    model = val[1].from_pretrained(key)

  trainer = Trainer(
      model=model, args=training_args, 
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
      compute_metrics=compute_metrics,
  )

  trainer.train()

  preds = trainer.predict(test_dataset=test_dataset)

  if key=='xlnet-base-cased':
    preds = trainer.predict(test_dataset)
    preds = preds.predictions[:, 0:2]
    results = np.argmax(preds, axis=-1)
  else:
    probs = torch.from_numpy(preds[0]).softmax(1)
    preds = probs.numpy()
    results = np.argmax(preds,axis=1)

  print(classification_report(test_labels, results))

  hard_predictions.append(results)
  soft_predictions.append(preds)
  

In [ ]:
binary_soft_predictions = (soft_predictions[0][:,:-1]+soft_predictions[1]+soft_predictions[2])/3
binary_hard_predictions = np.array(np.mean(np.array(hard_predictions), axis=0) > 0.5).astype(int)

print(classification_report(test_labels, np.argmax(binary_soft_predictions, axis=1)))
print(classification_report(test_labels, binary_hard_predictions))

## Explainability

Done using https://github.com/hila-chefer/Transformer-Explainability and the Captum package

In [ ]:
!git clone https://github.com/hila-chefer/Transformer-Explainability.git

import os
os.chdir(f'./Transformer-Explainability')

Cloning into 'Transformer-Explainability'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 386 (delta 3), reused 2 (delta 0), pack-reused 377
Receiving objects: 100% (386/386), 3.85 MiB | 13.36 MiB/s, done.
Resolving deltas: 100% (194/194), done.


In [ ]:
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import (
    visualization
)
import torch

In [ ]:
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import BertPreTrainedModel
from BERT_explainability.modules.layers_ours import *
from BERT_explainability.modules.BERT.BERT import BertModel
from torch.nn import CrossEntropyLoss, MSELoss

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = Dropout(config.hidden_dropout_prob)
        self.classifier = Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def relprop(self, cam=None, **kwargs):
        cam = self.classifier.relprop(cam, **kwargs)
        cam = self.dropout.relprop(cam, **kwargs)
        cam = self.bert.relprop(cam, **kwargs)
        # print("conservation: ", cam.sum())
        return cam

In [ ]:
# use bert-base model and tokenizer fine-tuned on our dataset without augmentation
model = model
tokenizer = tokenizer
model.eval()
explanations = Generator(model)

In [ ]:
text_batch = test_tweets[0]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].cuda()
attention_mask = encoding['attention_mask'].cuda()

true_class = 0

expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
expl = (expl - expl.min()) / (expl.max() - expl.min())

output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()

classifications = ["1", "2"]
class_name = classifications[classification]

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class, 1, tokens, 1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('size', 0.12280241400003433), ('on', 0.12573294341564178), ('the', 0.05040403828024864), ('the', 0.030341923236846924), ('toulouse', 0.04929311200976372), ('team', 0.04048938676714897), (',', 0.18840965628623962), ('that', 0.24687208235263824), ('pack', 0.08874382823705673), ('is', 0.3363882303237915), ('mons', 1.0), ('##tro', 0.29786866903305054), ('##us', 0.6187840104103088), ('.', 0.34725049138069153), ('i', 0.05266257002949715), ('can', 0.0427778922021389), ("'", 0.020743733271956444), ('t', 0.026094427332282066), ('see', 0.0), ('a', 0.008165671490132809), ('welsh', 0.03817109018564224), ('region', 0.00046567944809794426), ('ever', 0.010123890824615955), ('winning', 0.006363356951624155), ('this', 0.05046335980296135), (',', 0.21455958485603333), ('money', 0.07296325266361237), ('talks', 0.06711722910404205), ('as', 0.23539891839027405), ('they', 0.03478177636861801), ('say', 0.02806495688855648), ('.', 0.18602493405342102), ('[SEP]', 0.02023129165172577)]


In [ ]:
text_batch = test_tweets[51]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].cuda()
attention_mask = encoding['attention_mask'].cuda()

true_class = 0

expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
expl = (expl - expl.min()) / (expl.max() - expl.min())

output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
class_name = classifications[classification]

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('we', 0.31147047877311707), ('need', 0.0), ('a', 0.20068085193634033), ('big', 0.9575868844985962), ('win', 0.1131214052438736), ('against', 0.39424678683280945), ('charlton', 1.0), ('next', 0.8191582560539246), ('weekend', 0.02106725051999092), ('[SEP]', 0.01705043762922287)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,3 (0.28),0,1.00,[CLS] we need a big win against charlton next weekend [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,3 (0.28),0,1.00,[CLS] we need a big win against charlton next weekend [SEP]


In [ ]:
#1
#Scores = [('[CLS]', 0.0), ('so', 0.3725809156894684), ('the', 0.17398375272750854), ('scottish', 0.13383816182613373), ('government', 0.07083307951688766), ('want', 0.1194242313504219), ('people', 0.06945335865020752), ('to', 0.022305114194750786), ('get', 0.09672229737043381), ('their', 0.03423971310257912), ('boost', 0.0), ('##er', 0.012416996993124485), ('shots', 0.06714650243520737), ('so', 0.5115399956703186), ('badly', 0.9102041721343994), ('that', 0.37313026189804077), ('the', 0.19795337319374084), ('website', 0.125137597322464), ('doesn', 1.0), ("'", 0.23679029941558838), ('t', 0.7906231880187988), ('even', 0.6365886330604553), ('work', 0.10039672255516052), ('[SEP]', 0.022343693301081657)]

#1
#Scores = wow = 1, so = 0.67, good = 0.38

#0
#Scores = big = 0.95, charlton = 1.0 